In [1]:
import arcpy
import os

In [3]:
# ===== 1. Configuración inicial =====
input_folder = r"C:\Users\zmrplaza\OneDrive - The University of Memphis\Classes\Spring 2025\GIS Lab\Project\shapes"
output_folder = os.path.join(input_folder, "Results")
proj_folder = os.path.join(input_folder, "Projected")

arcpy.env.overwriteOutput = True
os.makedirs(output_folder, exist_ok=True)
os.makedirs(proj_folder, exist_ok=True)

# ===== 2. Datos =====
shapefiles = ["Shelby_County.shp", "chronic_normalized_tracts.shp", "pct_u5.shp", "Shelby_Renter_Percent.shp"]
value_fields = ["RPL_THEMES", "ghlth_crud", "pct_u5", "pct_renter"]
raster_outputs = ["r1.tif", "r2.tif", "r3.tif", "r4.tif"]

weights = [0.4125, 0.2626, 0.1625, 0.1625]

# Proyección: UTM Zone 16N (metros)
crs_target = arcpy.SpatialReference(26916)

# =========================================
# ETAPA 1: REPROYECTAR Y RASTERIZAR SIN PONDERACIÓN
# =========================================

print("🔹 Iniciando reproyección y rasterización...")

projected_shapefiles = []
for shp in shapefiles:
    in_shp = os.path.join(input_folder, shp)
    out_shp = os.path.join(proj_folder, "proj_" + shp)
    arcpy.management.Project(in_shp, out_shp, crs_target)
    projected_shapefiles.append(out_shp)

print("✅ Shapefiles reproyectados.")

# Definir extensión y resolución
shelby_proj = projected_shapefiles[0]
desc = arcpy.Describe(shelby_proj)
arcpy.env.extent = desc.extent
arcpy.env.cellSize = 10  # Resolución de 10 metros

print(f"🔧 Extensión fijada a Shelby County: {arcpy.env.extent}")
print("🔧 Resolución fijada a 10 metro.")

# Rasterizar cada shapefile sin aplicar pesos
rasters_paths = []
for shp, field, raster_name in zip(projected_shapefiles, value_fields, raster_outputs):
    raster_path = os.path.join(output_folder, raster_name)
    
    arcpy.conversion.PolygonToRaster(
        in_features=shp,
        value_field=field,
        out_rasterdataset=raster_path,
        cell_assignment="MAXIMUM_COMBINED_AREA"
    )
    rasters_paths.append(raster_path)

print("✅ Rasterización completa. Raster guardados sin ponderación.")

# =========================================
# ETAPA 2: ÁLGEBRA DE MAPAS CON PONDERACIÓN
# =========================================

print("🔹 Iniciando álgebra de mapas...")

# Leer los raster generados
rasters = [arcpy.Raster(r) for r in rasters_paths]

# Tratar NoData (convertimos solo los vacíos a 0)
rasters_clean = [arcpy.sa.Con(arcpy.sa.IsNull(r), 0, r) for r in rasters]

# Aplicar fórmula de ponderación
svi_heat = sum(r * w for r, w in zip(rasters_clean, weights))

# Guardar el raster final
svi_output = os.path.join(output_folder, "SVI_Heat_10m.tif")
svi_heat.save(svi_output)

print(f"🎯 Álgebra completada. Raster final guardado en: {svi_output}")


🔹 Iniciando reproyección y rasterización...
✅ Shapefiles reproyectados.
🔧 Extensión fijada a Shelby County: 197867.663030358 3875715.6571347 260830.485971837 3921826.41902268 NaN NaN NaN NaN
🔧 Resolución fijada a 10 metro.
✅ Rasterización completa. Raster guardados sin ponderación.
🔹 Iniciando álgebra de mapas...
🎯 Álgebra completada. Raster final guardado en: C:\Users\zmrplaza\OneDrive - The University of Memphis\Classes\Spring 2025\GIS Lab\Project\shapes\Results\SVI_Heat_10m.tif
